In [1]:
import streamlit as st
from streamlit_quill import st_quill
from langchain.callbacks.base import BaseCallbackHandler
import re

from langchain_community.llms import Ollama
from langchain_ibm import WatsonxLLM

from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
import os

import debugpy
import streamlit as st
from langchain.callbacks import StreamlitCallbackHandler

In [2]:
debug =0
if debug:
    # pylint: disable=invalid-name
    markdown = st.markdown(
    """
    ## Ready to attach the VS Code Debugger!
    ![Python: Remote Attach](https://awesome-streamlit.readthedocs.io/en/latest/_images/vscode_python_remote_attach.png)
    for more info see the [VS Code section at awesome-streamlit.readthedocs.io]
    (https://awesome-streamlit.readthedocs.io/en/latest/vscode.html#integrated-debugging)
    """
    )

    if not debugpy.is_client_connected():
        debugpy.listen(5679)
        debugpy.wait_for_client()

    markdown.empty()


In [3]:
class StreamDisplayHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text="", display_method='markdown'):
        self.container = container
        self.text = initial_text
        self.display_method = display_method
        self.new_sentence = ""

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.new_sentence += token

        display_function = getattr(self.container, self.display_method, None)
        if display_function is not None:
            display_function(self.text)
        else:
            raise ValueError(f"Invalid display_method: {self.display_method}")

    def on_llm_end(self, response, **kwargs) -> None:
        self.text=""


In [4]:
mood="{mood}"
actor="{actor}"
instruction="{instruction}"
input_1="{input_1}"

objective_instruction="{objective_instruction}"
input_2="{input_2}"

output_instruction="{output_instruction}"
template="{template}"

language_target="{language_target}"

if 'response' not in st.session_state:
    st.session_state['response'] = ""

if 'revisions' not in st.session_state:
    st.session_state.revisions = []


2024-07-16 12:44:00.144 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


In [5]:
# Example TEMPLATES
TEMPLATES = {
    "Language Model": {
        "Prompt Template":f"""
### INSTRUCTION
{mood}
{actor}
{instruction}
```
{input_1}
```

{objective_instruction}
```
{input_2}
```

{output_instruction}
```
{template}
```

{language_target} {actor}
### RESPONSE
"""
    },
    "First Touch": {
        "Email": """Thank you for contacting Red Hat about the issue you're experiencing with your [system/service/etc.]. My name is Chen and I'm here to help you resolve this issue as quickly as possible.
I understand that [issue the customer is experiencing] can be frustrating and I'm committed to assisting you in any way I can. As a first step, you might try [suggested action plan to resolve the issue]. If that doesn't work, please let me know and we can explore other options together.
Please let me know if you have any further questions or if there's anything else I can do to help. Thank you for choosing Red Hat for your support needs. We value your business and strive to provide the best service possible.
Best Regards,""",
        "Instruction": "Write a client facing email in response to this email:",
        "Objective": "The email needs to convey the following objective:",
        "Input_Context": "Case Description",
        "Input_Objective": "Response Objective",
        "Output_Format": "Using this as a template:",
        "Execution Code": f"instruction=Instruction,objective_instruction=Objective,output_instruction=Output_Format,input_1=input_1,input_2=input_2,template=Email,language_target=language_target,mood=mood,actor=actor"
    },
    "Case Continuation": {
        "Email": """Thank you for getting back to us,
[EMAIL_BODY]
Please let me know if you have any further questions or if there's anything else I can do to help. Thank you for choosing Red Hat for your support needs. We value your business and strive to provide the best service possible.
Best Regards,""",
        "Instruction": "Write a client facing email in response to this email:",
        "Objective": "The email needs to convey the following objective:",
        "Input_Context": "Case Description",
        "Input_Objective": "Response Objective",
        "Output_Format": "Using this as a template:",
        "Execution Code": f"instruction=Instruction,objective_instruction=Objective,output_instruction=Output_Format,input_1=input_1,input_2=input_2,template=Email,language_target=language_target,mood=mood,actor=actor"
   
    },
    "Assisting Colleague": {
        "Email": """My name is Chen and I will also be helping you resolve this issue along with my colleagues.
[EMAIL_BODY]
Please let me know if you have any further questions or if there's anything else I can do to help. Thank you for choosing Red Hat for your support needs. We value your business and strive to provide the best service possible.
Best Regards,""",
        "Instruction": "Write a client facing email in response to this email:",
        "Objective": "The email needs to convey the following objective:",
        "Input_Context": "Case Description",
        "Input_Objective": "Response Objective",
        "Output_Format": "Using this as a template:",
        "Execution Code": f"instruction=Instruction,objective_instruction=Objective,output_instruction=Output_Format,input_1=input_1,input_2=input_2,template=Email,language_target=language_target,mood=mood,actor=actor"
   
    },
}

In [6]:
# Function to handle template selection and content retrieval
def get_template_content(set_name, template_name):
    return TEMPLATES[set_name][template_name]


def prompt_template_maker(template,data):
    prompt=get_template_content("Language Model","Prompt Template")
    assignments = data.split(',')
    variables = {}
    for assignment in assignments:
        print(assignment)
        key, value = assignment.split('=')
        if value.strip() in TEMPLATES[template]:
            globals()[key.strip()] = get_template_content(template, value.strip())
            variables[key] = get_template_content(template, value)
        else:
            globals()[key.strip()] =globals()[value.strip()]
            variables[key] = globals()[value.strip()]

            
    prompt_template = PromptTemplate.from_template(prompt)
    prompt_template.partial(**variables)
    return prompt_template

def prompt_run_maker(template,data):
    prompt_template=get_template_content("Language Model","Prompt Template")
    assignments = data.split(',')
    variables = {}
    for assignment in assignments:
        print(assignment)
        key, value = assignment.split('=')
        if value.strip() in TEMPLATES[template]:
            globals()[key.strip()] = get_template_content(template, value.strip())
            variables[key] = get_template_content(template, value)
        else:
            globals()[key.strip()] =globals()[value.strip()]
            variables[key] = globals()[value.strip()]
    if actor_sel !="Normal":
        print ("Parmaters Overwritten")
        for param_override in actor_params:
            variables[param_override] = actor_params[param_override]

    return variables



In [7]:
ollama = Ollama(base_url="http://localhost:11434", model="autopilot")